In [ ]:
import pulp
from pulp import LpVariable, lpSum, LpAffineExpression
from collections.abc import Iterable
from itertools import product

In [ ]:
image_len = 28
n_layer_neurons = (image_len*image_len, 10, 10) # [28*28, 100, 10] default
layer_shapes = ((image_len,image_len), (n_layer_neurons[-2], 1), (n_layer_neurons[-1], 1))
num_steps = 5

In [ ]:
from mnist import MNIST
import numpy as np

def load_mnist():
    # Parameter setting
    GrayLevels = 255  # Image GrayLevels
    cats = [*range(10)]

    # General variables
    images = []  # To keep training images
    labels = []  # To keep training labels
    images_test = []  # To keep test images
    labels_test = []  # To keep test labels

    # loading MNIST dataset
    mndata = MNIST("data/mnist/MNIST/raw/")

    Images, Labels = mndata.load_training()
    Images = np.array(Images)
    for i in range(len(Labels)):
        if Labels[i] in cats:
            images.append(
                np.floor(
                    (GrayLevels - Images[i].reshape(28, 28))
                    * (num_steps - 1)
                    / GrayLevels
                ).astype(int)
            )
            labels.append(cats.index(Labels[i]))
    Images, Labels = mndata.load_testing()
    Images = np.array(Images)
    for i in range(len(Labels)):
        if Labels[i] in cats:
            images_test.append(
                np.floor(
                    (GrayLevels - Images[i].reshape(28, 28))
                    * (num_steps - 1)
                    / GrayLevels
                ).astype(int)
            )
            labels_test.append(cats.index(Labels[i]))

    del Images, Labels

    return images, labels, images_test, labels_test

In [ ]:
from utils.mnist_net import prepare_weights
weights_list = prepare_weights(subtype="mnist", load_data_func=load_mnist)

In [ ]:
theta = 1.0  # spike threshold
tau = 1  # synaptic delay

model = pulp.LpProblem("MultiLayer_SNN_Verification", pulp.LpMinimize)
M = 1000000  # Large constant for MILP
EPS = 1e-5

def get_layer_neurons_iter(layer: int) -> Iterable[tuple[int, int]]:
    match layer:
        case 0:
            return product(range(10,21,5),range(10,21,5))
        case _:
            return product(range(layer_shapes[layer][0]), range(layer_shapes[layer][1]))
s0_orig = np.random.randint(0, num_steps, size=layer_shapes[0])

# Variables: spike time and perturbation (input layer)
spike_times = dict()  # s[l,n] = spike time
neuron_perturbation = dict()  # d_n = perturbation for neuron n
for neuron in get_layer_neurons_iter(0):
    spike_times[neuron, 0] = LpVariable(f"s_0_{neuron}", 0, num_steps - 1, cat=pulp.LpInteger) # Xi_1
    ## Begin Xi_7
    # neuron_perturbation[neuron] = LpVariable(f"d_{neuron}", 0, num_steps - 1, cat=pulp.LpInteger)
    # model += spike_times[neuron, 0] - s0_orig[neuron] <= neuron_perturbation[neuron]
    # model += s0_orig[neuron] - spike_times[neuron, 0] <= neuron_perturbation[neuron]
    spike_times[neuron, 0] = s0_orig[neuron]
    ## End Xi_7
# model += lpSum(neuron_perturbation.values()) <= delta

# Intermediate variables
p = dict()  # p[l,t,n] = potential at layer l, time t, neuron n
flag = dict() # a[l,t,n] = activation flag for neuron n at layer l, time t
activated = dict()  # (p[l,t,n] >= threshold)
cond = dict()  # cond[l,t,n] = If (s_{l,n} ≤ t, 1, 0)

# Variables and constraints for each layer ≥ 1
for post_layer in range(1, len(n_layer_neurons)):
    for post_neuron in get_layer_neurons_iter(post_layer):
        assert tau * post_layer <= num_steps - 1, "Too high synaptic delay."
        spike_times[post_neuron, post_layer] = LpVariable(f"s_{post_neuron}_{post_layer}", tau * post_layer, num_steps - 1, cat=pulp.LpInteger) # Xi_1
        for t in range(num_steps):
            flag[post_neuron, post_layer, t] = LpVariable(f"a_{post_neuron}_{post_layer}_{t}", cat=pulp.LpBinary)
            activated[post_neuron, post_layer, t] = LpVariable(f"(p>=theta)_{post_neuron}_{post_layer}_{t}", cat=pulp.LpBinary)

# Condition variables for previous layers, used in Xi_3
for prev_layer in range(len(n_layer_neurons) - 1):
    for prev_neuron in get_layer_neurons_iter(prev_layer):
        _spike_time = spike_times[prev_neuron, prev_layer]
        for t in range(num_steps):
            _cond = cond[prev_neuron, prev_layer, t] = LpVariable(f"If_{prev_neuron}_{prev_layer}_{t}", cat=pulp.LpBinary)
            model += t + (1 - _cond) * M >= _spike_time
            model += t + 1 - _cond * M <= _spike_time
            # model += _spike_time >= t - tau + EPS - _cond * M

# Potential accumulation and spike decision
for post_layer in range(1, len(n_layer_neurons)):
    prev_layer = post_layer - 1
    for post_neuron in get_layer_neurons_iter(post_layer):
        p[post_neuron, post_layer, 0] = lpSum([])  # Xi_2
        for t in range(1, num_steps):
            ### Begin Xi_3
            expr = list[LpAffineExpression]()
            for prev_neuron in get_layer_neurons_iter(prev_layer):
                weight = weights_list[prev_layer][post_neuron[0], prev_neuron[0], prev_neuron[1]]
                expr.append(weight * cond[prev_neuron, prev_layer, t-tau])
            p[post_neuron, post_layer, t] = lpSum(expr)
            ### End Xi_3
            
        ### Begin Xi_4
        # Big-M method for spike condition
        model += flag[post_neuron, post_layer, 0] == 0  # Initial activation flag
        for t in range(1, num_steps):
            _flag = flag[post_neuron, post_layer, t]
            expr = list[LpVariable]()
            for t_prev in range(t):
                _activated = activated[post_neuron, post_layer, t_prev]
                model += p[post_neuron, post_layer, t] <= theta - EPS + _activated * M # Not active
                model += p[post_neuron, post_layer, t] >= theta - (1 - _activated) * M # Active
                model += _flag >= _activated
                expr.append(_activated)
            model += _flag <= lpSum(expr)
        ### End Xi_4

        # ### Begin Xi_5, Xi_6
        # spike_cond = dict[int, LpVariable]()   
        # for t in range(tau * post_layer, num_steps - 1):
        #     _spike_cond = spike_cond[t] = LpVariable(f"xi_5_lhs_{post_layer}_{t}_{post_neuron}", cat=pulp.LpBinary)
        #     model += _spike_cond <= 1 - flag[post_neuron, post_layer, t]
        #     model += _spike_cond <= activated[post_neuron, post_layer, t]
        #     model += _spike_cond >= (1 - flag[post_neuron, post_layer, t]) + activated[post_neuron, post_layer, t] - 1
        # xi_5_term = lpSum([t * spike_cond[t] for t in range(tau * post_layer, num_steps - 1)])
        # xi_6_term = (num_steps - 1) * (1 - flag[post_neuron, post_layer, num_steps - 1])
        # model += spike_times[post_neuron, post_layer] ==  xi_5_term + xi_6_term
        # ### End Xi_5, Xi_6

target_spike_time = spike_times[(7, 0), len(n_layer_neurons) - 1]

# ### Begin Xi_8
# # Robustness constraint: output neuron 1 should not spike earlier than neuron 0
# not_robust = list[LpVariable]()
# for out_neuron in get_layer_neurons_iter(len(n_layer_neurons) - 1):
#     if out_neuron[0] == pred_orig: continue
    
#     _other_spike_time = spike_times[out_neuron, len(n_layer_neurons) - 1]
#     # Ensure that output neuron 1 spikes at least 1 time step after output neuron 0
#     _not_robust = LpVariable(f"not_robust_{out_neuron}", cat=pulp.LpBinary)
#     model += _other_spike_time <= target_spike_time + (1 - _not_robust) * M
#     model += _other_spike_time >= target_spike_time + EPS - _not_robust * M
#     not_robust.append(_not_robust)
# model += lpSum(not_robust) >= 1  # Xi_8
# ### End Xi_8

# Dummy objective
model += target_spike_time

# Solve
solver = pulp.PULP_CBC_CMD(msg=True, logPath="log/milp.log")
model.solve(solver)

A:
MINIMIZE
1*k + 0
SUBJECT TO
_C1: k >= 2

_C2: k <= 2

VARIABLES
0 <= k <= 3 Integer

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/conda/envs/smt312/lib/python3.12/site-packages/pulp/apis/../solverdir/cbc/linux/i64/cbc /tmp/e173d30313a54bbaa9b901df27afbf48-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/e173d30313a54bbaa9b901df27afbf48-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 7 COLUMNS
At line 13 RHS
At line 16 BOUNDS
At line 18 ENDATA
Problem MODEL has 2 rows, 1 columns and 2 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 2 - 0.00 seconds
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cbc3007W No integer variables - nothing to do
Cuts at root node changed objective from 2 to -1.79769e+308
Probing was tried 0 times and created 0 cuts of which 0 were a